# Classification Experiment on AMI-Br

In this experiment, we split the data up according to patients before doing the train/val/test split.

This is to set a baseline for future experiments.

The pipeline uses only basic augmentation and trials two architectures:
- EfficientNetV2-S
- DenseNet121
  
and three strategies:
- weighted sampling (similar to minority oversampling with augmentation)
- weighted loss (cross-entropy)
- focal loss



In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.utils.data import Subset
from torchvision.ops import sigmoid_focal_loss


from dataset import AMFDataset

import albumentations as A
import numpy as np
from tiatoolbox.tools.stainaugment import StainAugmentor
from albumentations.pytorch import ToTensorV2

stain_matrix = np.array([[0.91633014, -0.20408072, -0.34451435],
               [0.17669817, 0.92528011, 0.33561059]])

aug_pipline = A.Compose([
                        A.RandomRotate90(),
                        A.HorizontalFlip(0.5),
                        A.VerticalFlip(0.5),
                        StainAugmentor(method='macenko', stain_matrix=stain_matrix),
                        A.RandomBrightnessContrast(p=0.2),
                        ToTensorV2(),
                        ])

nonaug_pipeline = A.Compose([
    ToTensorV2(),  # Convert to tensor
])


/home/aubreville/venv/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.24 (you have 1.4.21). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
# import the library
import wandb


In [3]:
import json
from torch import nn
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import balanced_accuracy_score, roc_auc_score
from torch.utils.data import WeightedRandomSampler
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import random_split

class EfficientNetClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(EfficientNetClassifier, self).__init__()
        # Load pre-trained EfficientNet_V2_S model
        self.efficientnet = models.efficientnet_v2_s(weights=models.efficientnet.EfficientNet_V2_S_Weights.DEFAULT)
        # Replace the classifier
        self.efficientnet.classifier = nn.Linear(list(self.efficientnet.classifier.modules())[-1].in_features, num_classes)

    def forward(self, x):
        return self.efficientnet(x)

class DenseNetClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(DenseNetClassifier, self).__init__()
        # Load pre-trained DenseNet model
        self.densenet = models.densenet121(weights=models.densenet.DenseNet121_Weights.DEFAULT)
        # Replace the classifier
        self.densenet.classifier = nn.Linear(self.densenet.classifier.in_features, num_classes)

    def forward(self, x):
        return self.densenet(x)

# retrieved from: https://github.com/clcarwin/focal_loss_pytorch/blob/master/focalloss.py
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()
            


def train_model(model, train_loader, val_loader, test_loader, criterion, optimizer, condition, num_epochs=10, device='cuda'):
    model.to(device)
    best_acc = 0

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        running_loss = 0.0
        running_corrects = 0
        total_samples = 0
        val_running_corrects = 0
        val_total_samples = 0
        all_labels = []
        all_preds = []
        all_probs = []  # To store predicted probabilities for AUC-ROC

        
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs.float()
            labels = labels.to(device).long()
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
        
            # Calculate loss
            loss = criterion(outputs, labels)
            
            # Backward pass + optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total_samples += labels.size(0)

        
        for inputs, labels in val_loader:
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs.float()
                labels = labels.to(device).long()
                
                
                # Forward pass
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                probs = torch.softmax(outputs, dim=1)[:, 1]
                
                val_running_corrects += torch.sum(preds == labels.data)
                val_total_samples += labels.size(0)

                all_labels.extend(labels.cpu().numpy())
                all_preds.extend(preds.cpu().numpy())
                all_probs.extend(probs.cpu().detach().numpy())  # Detach probabilities for AUC-ROC

        balanced_acc = balanced_accuracy_score(all_labels, all_preds)
        auc_roc = roc_auc_score(all_labels, all_probs)
        if balanced_acc>best_acc:
            torch.save(model.state_dict(), f'best_model.pt')
            print('New best model found.')
            best_acc=balanced_acc

        epoch_loss = running_loss / total_samples
        epoch_acc = running_corrects.to(torch.device('cpu')).double() / total_samples
        val_epoch_acc = val_running_corrects.to(torch.device('cpu')).double() / val_total_samples
        wandb.log({'training_loss': running_loss,
                   'validation_acc' : val_epoch_acc,
                   'validation_balanced_acc': balanced_acc,
                   'validation_auc' : auc_roc})
        
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}, Val Accuracy {val_epoch_acc:.4f}, balanced_acc: {balanced_acc:.4f}, roc_auc_score: {auc_roc:.4f} ')
    
    all_labels = []
    all_preds = []
    all_probs = []  # To store predicted probabilities for AUC-ROC

    model.load_state_dict(torch.load('best_model.pt', weights_only=True))
    for inputs, labels in test_loader:
        with torch.no_grad():
            inputs, labels = inputs.to(device).float(), labels.to(device).long()
            

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            probs = torch.softmax(outputs, dim=1)[:, 1]
            
            val_running_corrects += torch.sum(preds == labels.data)
            val_total_samples += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().detach().numpy())  # Detach probabilities for AUC-ROC    
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    auc_roc = roc_auc_score(all_labels, all_probs)
    log = { 'run' : epoch,
            'val_max_balanced_acc' : best_acc,
            'test_balanced_acc': balanced_acc,
            'test_auc' : auc_roc}
    try:
        wandb.log(log)
    except:
        pass

    print(f'TEST: balanced_acc: {balanced_acc:.4f}, roc_auc_score: {auc_roc:.4f}')
    f = open(f'classification_results_{condition}.log','a')
    f.write(json.dumps(log)+'\n')
    f.close()
    
    return model, all_probs, all_labels, all_preds

import pandas as pd
from sklearn.model_selection import train_test_split

def main(num_epochs=30, arch='EfficientNetV2', strategy='weightedsampling', learning_rate=0.001, class_weights=None):
    amibr_df = pd.read_csv('AMI-BR.csv')
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Train the model
    for k in range(5):
        run = wandb.init(
            project="AMI-Br", 
            name=f"run_{k}_{arch}_{strategy}", 
            id=f"run_{k}_{arch}_{strategy}",
        )            
        print('5-fold-MC-CV, Run: ',k)




        files_train, files_validtest = train_test_split(amibr_df.slide.unique(), train_size=0.7)
        files_valid, files_test = train_test_split(files_validtest, train_size=0.5)


        idxs_train = [x in files_train for x in amibr_df.slide.to_list()]
        ids_train = amibr_df.uid[idxs_train].to_list()
        dataset_train = amibr_df.dataset[idxs_train].to_list()
        fileid_train = [str(d)+'_'+str(i)+'.png' for d,i in zip(dataset_train,ids_train)]
        

        idxs_valid = [x in files_valid for x in amibr_df.slide.to_list()]
        ids_valid = amibr_df.uid[idxs_valid].to_list()
        dataset_valid = amibr_df.dataset[idxs_valid].to_list()
        fileid_valid = [str(d)+'_'+str(i)+'.png' for d,i in zip(dataset_valid,ids_valid)]

        idxs_test = [x in files_test for x in amibr_df.slide.to_list()]
        ids_test = amibr_df.uid[idxs_test].to_list()
        dataset_test = amibr_df.dataset[idxs_test].to_list()
        fileid_test = [str(d)+'_'+str(i)+'.png' for d,i in zip(dataset_test,ids_test)]


        # Create dataset instance
        dataset_train = AMFDataset(root_dir='patches', transform=aug_pipline, ids=fileid_train)
        dataset_valid = AMFDataset(root_dir='patches', transform=nonaug_pipeline, ids=fileid_valid)
        dataset_test = AMFDataset(root_dir='patches', transform=nonaug_pipeline, ids=fileid_test)

        print('TRAINING OVERVIEW:\n')
        print('==================\n')
        print('Number of patients in train: ',len(fileid_train),'(%.2f percent)' % (100*len(files_train)/len(amibr_df.slide.unique())))
        print('Number of patients in valid: ',len(fileid_valid),'(%.2f percent)' % (100*len(files_valid)/len(amibr_df.slide.unique())))
        print('Number of patients in test: ',len(fileid_test),'(%.2f percent)' % (100*len(files_test)/len(amibr_df.slide.unique())))

        print('Number of cells in train: ',len(dataset_train),'(%.2f percent)' % (100*len(dataset_train)/len(amibr_df)))
        print('Number of cells in valid: ',len(dataset_valid),'(%.2f percent)' % (100*len(dataset_valid)/len(amibr_df)))
        print('Number of cells in test: ',len(dataset_test),'(%.2f percent)' % (100*len(dataset_test)/len(amibr_df)))


        # Sampler for minority class undersampling
        
        train_labels = [y for x,y in dataset_train.images]
        count_amf = train_labels.count('atypical')
        count_nmf = train_labels.count('normal')
        weights = [1/count_amf if (x=='atypical') else 1/count_nmf for x in train_labels]

        sampler = WeightedRandomSampler(weights=weights, num_samples=len(train_labels), replacement=True)

        # Create dataloader
        if (strategy=='weightedsampling'):
            dataloader_train = DataLoader(dataset_train, batch_size=32, num_workers=8, sampler=sampler)
            criterion = nn.CrossEntropyLoss()
        elif (strategy=='weightedloss'):
            dataloader_train = DataLoader(dataset_train, batch_size=32, num_workers=8, shuffle=True)
            class_weights = [0.776, 0.224]  # Weights to reflect the dataset distribution
            weights = torch.tensor(class_weights, dtype=torch.float).to(device)
            criterion = nn.CrossEntropyLoss(weight=weights)
        elif (strategy=='focalloss'):
            dataloader_train = DataLoader(dataset_train, batch_size=32, num_workers=8, shuffle=True)
            criterion = FocalLoss()

            

        dataloader_valid = DataLoader(dataset_valid, batch_size=32, shuffle=False)
        dataloader_test = DataLoader(dataset_test, batch_size=32, shuffle=False)




        # Initialize model, loss, optimizer
        if (arch == 'EfficientNetV2'):
            model = EfficientNetClassifier(num_classes=2).to(device)
        elif (arch == 'DenseNet121'):
            model = DenseNetClassifier(num_classes=2).to(device)
        else:
            raise ValueError('please specify valid architecture')

        
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

        trained_model,all_probs, all_labels, all_preds = train_model(model, dataloader_train, dataloader_valid, dataloader_test, criterion, optimizer, condition=arch+'_'+strategy, num_epochs=num_epochs, device=device)
        run.finish()
    return trained_model,all_probs, all_labels, all_preds



In [4]:
trained_model,all_probs, all_labels, all_preds = main(num_epochs=100, arch='DenseNet121', strategy='weightedsampling', learning_rate=0.001)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: maubreville (muromi). Use `wandb login --relogin` to force relogin


5-fold-MC-CV, Run:  0
['atypical', 'normal']
['atypical', 'normal']
['atypical', 'normal']
TRAINING OVERVIEW:


Number of patients in train:  2408 (69.80 percent)
Number of patients in valid:  752 (14.85 percent)
Number of patients in test:  560 (15.35 percent)
Number of cells in train:  2408 (64.73 percent)
Number of cells in valid:  752 (20.22 percent)
Number of cells in test:  560 (15.05 percent)



00%|███████████████████████████████████████████| 76/76 [00:07<00:00, 10.21it/s]

New best model found.
Epoch 0/99, Loss: 0.6727, Accuracy: 0.6325, Val Accuracy 0.5093, balanced_acc: 0.4885, roc_auc_score: 0.5032 



00%|███████████████████████████████████████████| 76/76 [00:07<00:00, 10.79it/s]

...


00%|███████████████████████████████████████████| 71/71 [00:09<00:00,  7.66it/s]

Epoch 98/99, Loss: 0.0762, Accuracy: 0.9739, Val Accuracy 0.6495, balanced_acc: 0.6386, roc_auc_score: 0.6290 



00%|███████████████████████████████████████████| 71/71 [00:08<00:00,  8.08it/s]

Epoch 99/99, Loss: 0.0760, Accuracy: 0.9717, Val Accuracy 0.6764, balanced_acc: 0.6501, roc_auc_score: 0.6492 
TEST: balanced_acc: 0.6657, roc_auc_score: 0.6642


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▇▇▆▆▆▆▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▁▂▂▂▁▂▂▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▃▁▆▅▇▆▅▆▆▆▆▆▅▅▅▆▆▆▆▇▆▆▆▆▇▆▇▇▆▇█▇▇▆█▇▇▇█▇
validation_auc,▂▂▁▂▂▄▂▃▂▂▄▃▃▄▄▂▃▄▄▄▅▅▆▅▇▇▆▆▇▇▅▆▆▇▆█▆█▆▇
validation_balanced_acc,▁▄▄▄▄▄▄▅▄▆▄▅▆▆▆▄▄▅▅▄▆▅▇▇▆▇▇▅▆▇▇▆█▆▇▆▇▇█▇
run,99
test_auc,0.66417
test_balanced_acc,0.66566


In [5]:
trained_model,all_probs, all_labels, all_preds = main(num_epochs=100, arch='EfficientNetV2', strategy='weightedsampling', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
['atypical', 'normal']
['atypical', 'normal']
TRAINING OVERVIEW:


Number of patients in train:  2360 (69.80 percent)
Number of patients in valid:  608 (14.85 percent)
Number of patients in test:  752 (15.35 percent)
Number of cells in train:  2360 (63.44 percent)
Number of cells in valid:  608 (16.34 percent)
Number of cells in test:  752 (20.22 percent)



00%|███████████████████████████████████████████| 74/74 [00:10<00:00,  7.03it/s]

New best model found.
Epoch 0/99, Loss: 0.5832, Accuracy: 0.6911, Val Accuracy 0.5066, balanced_acc: 0.5722, roc_auc_score: 0.5545 



00%|███████████████████████████████████████████| 74/74 [00:12<00:00,  6.10it/s]

New best model found.
Epoch 1/99, Loss: 0.4858, Accuracy: 0.7784, Val Accuracy 0.5740, balanced_acc: 0.5785, roc_auc_score: 0.5946 


...


00%|███████████████████████████████████████████| 74/74 [00:09<00:00,  7.53it/s]

Epoch 98/99, Loss: 0.0612, Accuracy: 0.9791, Val Accuracy 0.7766, balanced_acc: 0.7218, roc_auc_score: 0.7555 



00%|███████████████████████████████████████████| 74/74 [00:09<00:00,  7.65it/s]

Epoch 99/99, Loss: 0.0424, Accuracy: 0.9851, Val Accuracy 0.7974, balanced_acc: 0.7351, roc_auc_score: 0.7512 
TEST: balanced_acc: 0.7179, roc_auc_score: 0.7216


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▅▅▄▄▃▃▄▃▃▃▂▂▂▂▂▂▂▁▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▁▁▂▃▄▅▅▅▆▆▆▆▅▆▆▇▆▇▇▇▇▇▆▇█▇▇▅▇▇▇▆▇▇███▇██
validation_auc,▁▂▂▂▂▄▄▃▄▅▃▅▅▄▅▅▅▄▅▅▇▆▆▆▇▅▆▆▇▅█▅▅█▅▅▇▇▇▇
validation_balanced_acc,▂▁▁▃▃▄▃▅▅▄▅▅▆▃▆▆▅▃▄▆▅▆▆▆▅▆▆▆▆▇▇▇▆▇▅▇▅█▇▇
run,99
test_auc,0.72155
test_balanced_acc,0.71789


In [6]:
trained_model = main(num_epochs=100, arch='DenseNet121', strategy='weightedloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
['atypical', 'normal']
['atypical', 'normal']
TRAINING OVERVIEW:


Number of patients in train:  2716 (69.80 percent)
Number of patients in valid:  324 (14.85 percent)
Number of patients in test:  680 (15.35 percent)
Number of cells in train:  2716 (73.01 percent)
Number of cells in valid:  324 (8.71 percent)
Number of cells in test:  680 (18.28 percent)



00%|███████████████████████████████████████████| 85/85 [00:11<00:00,  7.69it/s]

New best model found.
Epoch 0/99, Loss: 0.7804, Accuracy: 0.5438, Val Accuracy 0.6821, balanced_acc: 0.4875, roc_auc_score: 0.4569 



00%|███████████████████████████████████████████| 85/85 [00:12<00:00,  6.72it/s]

New best model found.
Epoch 1/99, Loss: 0.7092, Accuracy: 0.5935, Val Accuracy 0.7160, balanced_acc: 0.5336, roc_auc_score: 0.5145 


...


00%|███████████████████████████████████████████| 66/66 [00:11<00:00,  5.67it/s]

Epoch 98/99, Loss: 0.1211, Accuracy: 0.9562, Val Accuracy 0.7418, balanced_acc: 0.6202, roc_auc_score: 0.6295 



00%|███████████████████████████████████████████| 66/66 [00:07<00:00,  8.90it/s]

Epoch 99/99, Loss: 0.1097, Accuracy: 0.9557, Val Accuracy 0.7167, balanced_acc: 0.6038, roc_auc_score: 0.6121 
TEST: balanced_acc: 0.6098, roc_auc_score: 0.6262


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▇▆▅▅▅▄▄▄▄▃▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁
val_max_balanced_acc,▁
validation_acc,█▁▄▃▄▃▄▄▅▅▅▅▆▅▅▅▅▆▄▆▄▄▆▆▆▅▆▅▅▅▆▅▇▆▇▆▆▆▆▇
validation_auc,▁▂▂▄▃▅▅▄▇▅▅▆▇▆▆▅▆▆▆▆█▆▆▇▇▇▇▇▇█▇▇████▇█▇▇
validation_balanced_acc,▁▁▃▄▄▅▃▄▄▅▆▆▆▆▅▅▆▅▅▅▅▅▆▆▇█▆▆▇▆▆▆▆▄▆▅▆▇▇█
run,99
test_auc,0.62623
test_balanced_acc,0.60978


In [7]:
trained_model = main(num_epochs=100, arch='EfficientNetV2', strategy='weightedloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
['atypical', 'normal']
['atypical', 'normal']
TRAINING OVERVIEW:


Number of patients in train:  2755 (69.80 percent)
Number of patients in valid:  514 (14.85 percent)
Number of patients in test:  451 (15.35 percent)
Number of cells in train:  2755 (74.06 percent)
Number of cells in valid:  514 (13.82 percent)
Number of cells in test:  451 (12.12 percent)



00%|███████████████████████████████████████████| 87/87 [00:11<00:00,  7.74it/s]

New best model found.
Epoch 0/99, Loss: 0.6545, Accuracy: 0.6592, Val Accuracy 0.6576, balanced_acc: 0.5366, roc_auc_score: 0.5178 



00%|███████████████████████████████████████████| 87/87 [00:11<00:00,  7.55it/s]

New best model found.
Epoch 1/99, Loss: 0.5443, Accuracy: 0.7623, Val Accuracy 0.6479, balanced_acc: 0.5462, roc_auc_score: 0.5290 


...


00%|███████████████████████████████████████████| 90/90 [00:14<00:00,  6.14it/s]

Epoch 98/99, Loss: 0.0837, Accuracy: 0.9730, Val Accuracy 0.7677, balanced_acc: 0.6972, roc_auc_score: 0.6728 



00%|███████████████████████████████████████████| 90/90 [00:13<00:00,  6.60it/s]

Epoch 99/99, Loss: 0.0895, Accuracy: 0.9663, Val Accuracy 0.7758, balanced_acc: 0.7187, roc_auc_score: 0.6950 
TEST: balanced_acc: 0.6445, roc_auc_score: 0.6291


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▇▆▆▆▅▅▅▅▄▄▄▃▃▃▃▃▂▂▃▂▂▂▂▁▁▂▂▂▂▂▁▁▂▁▁▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▂▁▂▃▃▃▃▄▄▅▅▆▅▆▆▄▆▇▆█▇▆▆▆▇▇██▇▇▇▇█▇▇█▇█▇█
validation_auc,▁▃▄▃▄▅▅▄▅▅▅▆▆▇▅▆▆▅▆▆▇▇▇▇▆▇█▇██▇█▇▇▇▇▇██▇
validation_balanced_acc,▁▁▂▂▃▃▃▄▅▅▄▆▆▅▆▆▆▇▆▇▆▆█▇▆▇▆▇██▇▆▆▇█▆▇▆▇█
run,99
test_auc,0.62912
test_balanced_acc,0.64447


In [8]:
trained_model = main(num_epochs=100, arch='DenseNet121', strategy='focalloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
['atypical', 'normal']
['atypical', 'normal']
TRAINING OVERVIEW:


Number of patients in train:  2805 (69.80 percent)
Number of patients in valid:  535 (14.85 percent)
Number of patients in test:  380 (15.35 percent)
Number of cells in train:  2805 (75.40 percent)
Number of cells in valid:  535 (14.38 percent)
Number of cells in test:  380 (10.22 percent)


|2025-01-03|17:53:01.055| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 88/88 [00:11<00:00,  7.73it/s]

New best model found.
Epoch 0/99, Loss: 0.1796, Accuracy: 0.7362, Val Accuracy 0.7458, balanced_acc: 0.5000, roc_auc_score: 0.5373 


|2025-01-03|17:53:14.845| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 88/88 [00:13<00:00,  6.49it/s]

New best model found.
Epoch 1/99, Loss: 0.1409, Accuracy: 0.7729, Val Accuracy 0.7290, balanced_acc: 0.5033, roc_auc_score: 0.4839 


...

|2025-01-03|19:58:36.977| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 88/88 [00:13<00:00,  6.75it/s]

Epoch 98/99, Loss: 0.0633, Accuracy: 0.8922, Val Accuracy 0.7316, balanced_acc: 0.5758, roc_auc_score: 0.6071 


|2025-01-03|19:58:52.895| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 88/88 [00:14<00:00,  6.07it/s]

Epoch 99/99, Loss: 0.0657, Accuracy: 0.8958, Val Accuracy 0.7347, balanced_acc: 0.5969, roc_auc_score: 0.6276 
TEST: balanced_acc: 0.5731, roc_auc_score: 0.6384


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▅▅▅▄▃▃▃▃▃▂▄▃▄▂▃▂▂▂▃▆▅▅▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁
val_max_balanced_acc,▁
validation_acc,▄▇█▇▆▇▁▇▄▄█▃▃▂▃▄▂█▆▅▄▃▄▃▄▃▅▃▄▃▃▅▇▆▄▅▄▆▄▅
validation_auc,▃▃▄▄▃▅▃▄▅▆▅▆▄▄▅▆▆▇▁▁▄▄▄▄▅▅▅▅▅▆▆▅▅▆▅▇▇█▆█
validation_balanced_acc,▁▁▃▃▄▁▄▄▅▃▄▅▆▇▄▄▅▇▆▄▂▁▃▂▇▄▄▄▅▆▆▆▇▆▆▇▆█▆▆
run,99
test_auc,0.63835
test_balanced_acc,0.57312


In [9]:
trained_model = main(num_epochs=100, arch='EfficientNetV2', strategy='focalloss', learning_rate=0.001)


5-fold-MC-CV, Run:  0
['atypical', 'normal']
['atypical', 'normal']
['atypical', 'normal']
TRAINING OVERVIEW:


Number of patients in train:  2596 (69.80 percent)
Number of patients in valid:  303 (14.85 percent)
Number of patients in test:  821 (15.35 percent)
Number of cells in train:  2596 (69.78 percent)
Number of cells in valid:  303 (8.15 percent)
Number of cells in test:  821 (22.07 percent)


|2025-01-03|19:59:18.580| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:10<00:00,  8.07it/s]

New best model found.
Epoch 0/99, Loss: 0.1281, Accuracy: 0.7747, Val Accuracy 0.7492, balanced_acc: 0.5197, roc_auc_score: 0.5813 


|2025-01-03|19:59:30.735| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 82/82 [00:13<00:00,  6.28it/s]

New best model found.
Epoch 1/99, Loss: 0.1063, Accuracy: 0.8163, Val Accuracy 0.6865, balanced_acc: 0.5445, roc_auc_score: 0.5666 


...

|2025-01-03|22:11:16.479| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 95/95 [00:15<00:00,  6.33it/s]

Epoch 98/99, Loss: 0.0176, Accuracy: 0.9784, Val Accuracy 0.7579, balanced_acc: 0.6572, roc_auc_score: 0.6743 


|2025-01-03|22:11:33.414| [WARNING] /tmp/ipykernel_872123/1051143625.py:53: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)


00%|███████████████████████████████████████████| 95/95 [00:14<00:00,  6.37it/s]

New best model found.
Epoch 99/99, Loss: 0.0145, Accuracy: 0.9857, Val Accuracy 0.7868, balanced_acc: 0.7045, roc_auc_score: 0.6579 
TEST: balanced_acc: 0.6792, roc_auc_score: 0.6575


run,▁
test_auc,▁
test_balanced_acc,▁
training_loss,█▇▆▆▆▅▄▅▄▄▄▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▂▁▁▁
val_max_balanced_acc,▁
validation_acc,▃▄▅▂▃▃▆▅▄▆▁▆▅▆▅▅▆▆▅▆▆▆▅▅▅▅▇▅▆▅▇▆▆▆▆▆▇▆▇█
validation_auc,▁▂▃▃▁▄▃▅▃▄▇▄▄▅▃▅▄▄▅▄▅▆▅▅▅▄▇▅▅▇█▅▇▅▇▇▇▅▆▆
validation_balanced_acc,▁▃▄▄▂▄▃▄▄▅▄▄▆▅▅▅▅▅▅▆▆▅▅▅▆▅▆▇▇▄▇▆▅▇▆▇▅▆▆█
run,99
test_auc,0.65746
test_balanced_acc,0.67924
